# This is my prototyping platform for the code to extract a timeseries of data from the NLDAS GRB files and store them in NetCDF format.
## There should be a .py script with a similar name that runs the finished code on HPC.

In [101]:
import datetime as dt
import numpy as np
import netCDF4 as nc # http://unidata.github.io/netcdf4-python/
import scipy as sp
import pygrib as pg
import numpy.ma as ma
import array as arr 
import xarray as xr

In [2]:
# Write the NetCDF forcing data file.
grib_dir = '/home/NearingLab/data/nldas/grib/NLDAS2.FORCING/'
write_dir = '/home/NearingLab/data/nldas/netcdf-single-cells/'

In [219]:
# Open an example file
fname = grib_dir + '1979/001/' + 'NLDAS_FORA0125_H.A19790101.1300.002.grb'
#fname = grib_dir + '2019/001/' + 'NLDAS_FORA0125_H.A20190101.0000.002.grb'
gbf = pg.open(fname)
lats = gbf[1].latitudes
lons = gbf[1].longitudes

In [74]:
# Open water mask
maskName = 'nldas_info/NLDAS_IGBPpredomveg.asc'
km2 = np.genfromtxt(maskName)[:,4]
waterMask = np.genfromtxt(maskName)[:,21]
for i in range(km2.shape[0]):
    if waterMask[i] > km2[i]/2:
        waterMask[i] = 'NaN'
    elif waterMask[i] <= km2[i]/2:
        waterMask[i] = 1
waterMask = np.reshape(waterMask,[224,464], order='A') #options are CFA
print(waterMask.shape)

(224, 464)


In [211]:
nrows = int(lats.shape[0]/waterMask.shape[1])
ncols= int(lons.shape[0]/waterMask.shape[0])

In [212]:
nrows

224

In [5]:
# Set start and end data information for the GRIB/NetCDF forcing data.
yearStart  = 1979
monthStart = 1 
dayStart   = 1 
hourStart  = 13
startDateTime = dt.datetime(yearStart, monthStart, dayStart, hour = hourStart)
print("Will be calculating hours starting from: ")
print(startDateTime)
dayOfYearStart = dt.datetime.date(startDateTime).timetuple().tm_yday
yearEnd  = 1980
monthEnd = 10
dayEnd   = 1 
hourEnd  = 12
endDateTime = dt.datetime(yearEnd, monthEnd, dayEnd, hour = hourEnd)
dayOfYearEnd = dt.datetime.date(endDateTime).timetuple().tm_yday

Will be calculating hours starting from: 
1979-01-01 13:00:00


In [6]:
# Initializing the directory, but will change each day and year.
mainDirectory = '/home/NearingLab/data/nldas/grib/NLDAS2.FORCING/'
startDirectory = mainDirectory + str(yearStart)  + "/" \
    + str("{:03d}".format(dayOfYearStart))  + "/"
endDirectory = mainDirectory + str(yearEnd)  + "/" \
    + str("{:03d}".format(dayOfYearEnd))  + "/"
filePrefix = 'NLDAS_FORA0125_H.A'
fileSufix = '.002.grb'

In [18]:
# specify the data and time to fine the correct file in this name format
startFileDateTime = dateForFile(yearStart, monthStart, dayStart, hourStart)
endFileDateTime = dateForFile(yearEnd, monthEnd, dayEnd, hourEnd)
#Add prefix and sufix to the date to create the whole file name.
startFile = getFileName(startFileDateTime, startDirectory, "A")
endFile = getFileName(endFileDateTime, endDirectory, "A")

In [182]:
# Need to get the GRIB time for the first and last files
#Start the loop at the first date in the files.
year1, month1, day1, hour1 = dateFromGRIB(startFile)
t = dt.datetime(year1, month1, day1, hour=hour1)
#Then have the loop run until the last file date.
year2, month2, day2, hour2 = dateFromGRIB(endFile)
endTime = dt.datetime(year2, month2, day2, hour=hour2)
# Set timestep to move forward, to run through the files
deltime = dt.timedelta(hours=1)
# Estimate the number of hours in the record
H = endTime - t # (t = startDateTime)
# Convert the time difference to hours) 
H = int(H.total_seconds()/60/60) + 1
time = [0 for x in range(H)]

In [20]:
# Make a list of all the times to loop through
dates = [startDateTime + deltime*h for h in range(H)]

In [171]:
llstr = 1000
llend = 1010
iloop = [x for x in range(llstr,llend)]
looplats = lats[iloop]
looplons = lons[iloop]
print(looplats)
print(looplons)

[25.313 25.313 25.313 25.313 25.313 25.313 25.313 25.313 25.313 25.313]
[-115.938 -115.813 -115.688 -115.563 -115.438 -115.313 -115.188 -115.063
 -114.938 -114.813]


In [239]:
G = {}
for x in iloop:
    for y in iloop:
        xy = "{}, {}".format(lats[x], -lons[y])
        G[xy] = setForcingLists(H)
gvars = {1:'airtemp', 2:'spechum', 3:'airpres', 4:'forcingUGRD', 5:'windspd',
    6:'LWRadAtm', 7:'forcingCONVfrac', 8:'forcingCAPE', 9:'forcingPEVAP', 10:'pptrate', 11:'SWRadAtm'}

In [184]:
np.set_printoptions(threshold=np.nan)

ValueError: threshold must be non-NAN, try sys.maxsize for untruncated representation

In [252]:
# Main loop through the GRIB files by one hour intervals. open, extract, write, save
# Main loop through the NetCDF files by one hour intervals. 
# iH: Index to use for filling forcing data list.
for iH, t in enumerate(dates):
    print("----------------------------------------------------")
    print("Current data & time in the main loop is (t): {}".format(t))

    hoursSinceStartDate = t - startDateTime
    hoursSinceStartDate = int(hoursSinceStartDate.total_seconds()/60/60)
    time[iH] = float(hoursSinceStartDate)
    print("Hours since the start date: {}".format(hoursSinceStartDate))

    # The files have both A and B versions.
    AB = "A"
    # Set the strings for the file name
    iYear, iMonth, iDay, iHour = getValuesFromDateTime(t)
    # Get the datetime stuff in strings to be used in the NetCDF file call.
    dateTime4File = dateForFile(iYear, iMonth, iDay, iHour)
    # Need to change the directory to reflect the loop data
    directory = changeDirectory(t)
    # Put the file name together, this includes the full path
    fileName = getFileName(dateTime4File, directory, AB)
    # Open the file for this particular data & time.
    try:
        grbForce = pg.open(fileName)
        # if the file exists, then advance the list index by one.
        iH = iH + 1
        # Store configuration file values
    except:
        # skip the file
        print('File not found:')
        print(fileName)
        continue
    # Find the index of the cell closest to the CAMELS latlon
    lats = grbForce[1].latitudes
    lons = grbForce[1].longitudes
    
    # Now loop across the grid in the X direction
    for x in range(464):
    
        # Now loop across the grid in the Y direction
        for y in range(224):
            xy = "{}, {}".format(lats[x], -lons[y])
            g = extractGrib(gbf, x, y)
            for iv, v in enumerate(gvars):
                G[xy][i].append(g[v])
                
            break
        break
    break

----------------------------------------------------
Current data & time in the main loop is (t): 1979-01-01 13:00:00
Hours since the start date: 0


KeyError: '25.063, 124.938'

In [247]:
G.keys()

dict_keys(['25.313, 115.938', '25.313, 115.813', '25.313, 115.688', '25.313, 115.563', '25.313, 115.438', '25.313, 115.313', '25.313, 115.188', '25.313, 115.063', '25.313, 114.938', '25.313, 114.813'])

In [ ]:
# Save the forcing data for each cell, individually
for x in iloop:
    for y in iloop:
        forcing = fillForcing(forcing, H, hruID, lat, lon, dt, time, 
                SWRadAtm, LWRadAtm, airpres, airtemp, pptrate, spechum, windspd)

# FUNCTIONS TO CALL IN THE MAIN LOOP

In [8]:
# Search for nearest decimal degree in an array of decimal degrees and return the index.
# np.argmin returns the indices of minimum value along an axis.
# so subtract dd from all values in dd_array, take absolute value and find index of minimum
def geo_idx(dd, dd_array):
    geo_idx = (np.abs(dd_array - dd)).argmin()
    return geo_idx

In [196]:
# Search and return the single index in the GRIB file with the lat/long found from geo_idx
def geo_idx_1(lat_val, lon_val, lats, lons):
    ilat = 0
    ilon = 0
    print("Looking for lat/lon index values")
    #Loop through the one dimensional latitude list,
    #Then we'll calculate where we should be in a two dimensional array
    for i in range(0, len(lats)):
        #At some point we should hit the SINGLE location in the 1D list...
        #Where the latitude and longitude values match our grid with our point of interest
        if lats[i] == lat_val and lons[i] == lon_val:
            #When this happens return those values, because we found our treasure
            return [ilat, ilon]
        #If for some reason we never find the treasure, let the user know.
        #And then end the loop before we get an error for the (i+1) index call
        if i == (len(lats)-1):
            print("ERROR: CAN NOT FIND THE INDEXIES FOR LATITUDE AND LONGITUDE!!!!!!!!")
            return [-99, -99]
            break
        #If latitudes reach the end of their cycle, then restart
        #The latitudes stay constant through the longitudes, then...
        #When the longitudes reach the minimum, the latitude moves down one.
        if lats[i] != lats[i + 1]:
            #Move on, because we've cycled through the longitudes...
            # associated with this latitude
            ilat = ilat + 1
            #The longitudes reset, so the index goes back to zero for the 2D array.
            ilon = 0
        else:
            #The longitudes keep moving while the latitude stays constant.
            ilon = ilon + 1


In [199]:
lat_val = 34.22223
lon_val = -118.17757
lat_val = geo_idx(lat_val, lats)
lon_val = geo_idx(lon_val, lons)
print(lat_val, lon_val)
print(lats[lat_val], lons[lon_val])
[lat_idx, lon_idx] = geo_idx_1(lat_val, lon_val, lats, lons)
print([lat_idx, lon_idx])
extractGrib(gbf, lat_idx, lon_idx)

33872 54
34.188 -118.188
Looking for lat/lon index values
ERROR: CAN NOT FIND THE INDEXIES FOR LATITUDE AND LONGITUDE!!!!!!!!
[-99, -99]


NameError: name 'grb' is not defined

In [10]:
def dateForFile(year, month, day, hour):
    # Set the strings for the file name
    yearStr = str("{:02d}".format(year))
    monthStr = str("{:02d}".format(month))
    dayStr = str("{:02d}".format(day))
    hourStr = str("{:02d}".format(hour))
    dateTime = yearStr + monthStr + dayStr + '.' +  hourStr + '00'
    return dateTime

In [11]:
def changeDirectory(t):
    year = dt.datetime.date(t).year
    day = dt.datetime.date(t).timetuple().tm_yday
    directory = grib_dir + str(year)  + "/" + str("{:03d}".format(day))  + "/"
    return directory

In [12]:
def getFileName(dateTime4File, directory, AB):
    if AB == "A":
        filePrefix = 'NLDAS_FORA0125_H.A'
    elif AB == "B":
        filePrefix = 'NLDAS_FORB0125_H.A'
    fileSufix = '.002.grb'
    fileName = directory + filePrefix + dateTime4File + fileSufix
    return fileName

In [13]:
def dateFromNetCDF(fileName):
    file_in = nc.Dataset(fileName,"r",format="NETCDF4")
    t_unit = file_in.variables["time"].units # get unit  "days since 1950-01-01T00:00:00Z"
    print("NetCDF file units for time are:")
    print(t_unit)
    year = int(t_unit[12:15+1])
    month = int(t_unit[17:18+1])
    day = int(t_unit[20:21+1])
    hour = int(t_unit[23:24+1])
    return t_unit, year, month, day, hour;

In [14]:
def dateFromGRIB(fileName):
    file_in = pg.open(fileName)
    gribData = file_in.select()[0]
    year = int(gribData.year)
    month = int(gribData.month)
    day = int(gribData.day)
    hour = int(gribData.hour)

    return year, month, day, hour;

In [15]:
def getValuesFromDateTime(t):
    y = dt.datetime.date(t).year
    m = dt.datetime.date(t).month
    d = dt.datetime.date(t).day
    h = dt.datetime.time(t).hour
    return y, m, d, h

In [16]:
def fillForcing(forcing, H, hruID, lat, lon, dt, time, \
                SWRadAtm, LWRadAtm, airpres, airtemp, pptrate, spechum, windspd):
    forcing.createDimension('hru', 1)
    forcing.createDimension('time', H)
    ### createVareables in new data set
    forcing.createVariable('hruId', np.int32, ('hru',))
    forcing.variables['hruId'].units = 'id number'
    forcing.variables['hruId'].long_name = 'The Hydrologic Response Unit identification number'
    forcing.createVariable('latitude', np.float32, ('hru',))
    forcing.variables['latitude'].units = 'decimal degree'
    forcing.variables['latitude'].long_name = 'Latitude location of HRU, North-South decimal degrees'
    forcing.createVariable('longitude', np.float32, ('hru',))
    forcing.variables['longitude'].units = 'decimal degree'
    forcing.variables['longitude'].long_name = 'Longitude location of HRU, East-West decimal degrees'
    forcing.createVariable('data_step', np.int32)
    forcing.variables['data_step'].units = 'seconds'
    forcing.variables['data_step'].long_name = 'data step length in seconds'
    forcing.createVariable('time', np.float64, ('time',))
    forcing.variables['time'].units = 'hours since 1979-01-01 00:00:00'
    forcing.variables['time'].long_name = 'time of forcing data'
    forcing.createVariable('LWRadAtm', np.float32, ('time', 'hru'))
    forcing.variables['LWRadAtm'].units = 'W m-2'
    forcing.variables['LWRadAtm'].long_name = 'downward longwave radiation at the upper boundary'
    forcing.variables['LWRadAtm'].v_type     = 'scalarv'
    forcing.createVariable('SWRadAtm', np.float32, ('time', 'hru'))
    forcing.variables['SWRadAtm'].units = 'W m-2'
    forcing.variables['SWRadAtm'].long_name = 'downward shortwave radiation at the upper boundary'
    forcing.variables['SWRadAtm'].v_type     = 'scalarv'
    forcing.createVariable('airpres', np.float32, ('time', 'hru'))
    forcing.variables['airpres'].units = 'Pa'
    forcing.variables['airpres'].long_name = 'air pressure at the measurement height'
    forcing.variables['airpres'].v_type     = 'scalarv'
    forcing.createVariable('airtemp', np.float32, ('time', 'hru'))
    forcing.variables['airtemp'].units = 'K'
    forcing.variables['airtemp'].long_name = 'air temperature at the measurement height'
    forcing.variables['airtemp'].v_type     = 'scalarv'
    forcing.createVariable('pptrate', np.float32, ('time', 'hru'))
    forcing.variables['pptrate'].units = 'kg m-2 s-1'
    forcing.variables['pptrate'].long_name = 'Precipitation rate'
    forcing.variables['pptrate'].v_type     = 'scalarv'
    forcing.createVariable('spechum', np.float32, ('time', 'hru'))
    forcing.variables['spechum'].units = 'g g-1'
    forcing.variables['spechum'].long_name = 'specific humidity at the measurement height'
    forcing.variables['spechum'].v_type     = 'scalarv'
    forcing.createVariable('windspd', np.float32, ('time', 'hru'))
    forcing.variables['windspd'].units = 'm s-1'
    forcing.variables['windspd'].long_name = 'wind speed at the measurement height'
    forcing.variables['windspd'].v_type     = 'scalarv'
    # Fill new data set with diplicate values
    forcing.variables['hruId'][:]          = hruID
    forcing.variables['latitude'][:]       = lat
    forcing.variables['longitude'][:]      = lon
    forcing.variables['data_step'][:]      = dt
    forcing.variables['time'][:]      = np.transpose(time)
    forcing.variables['SWRadAtm'][:]  = np.transpose(SWRadAtm)
    forcing.variables['LWRadAtm'][:]  = np.transpose(LWRadAtm)
    forcing.variables['airpres'][:]   = np.transpose(airpres)
    forcing.variables['airtemp'][:]   = np.transpose(airtemp)
    forcing.variables['pptrate'][:]   = np.transpose(pptrate)
    forcing.variables['spechum'][:]   = np.transpose(spechum)
    forcing.variables['windspd'][:]   = np.transpose(windspd)

    return forcing

In [17]:
def setForcingLists(H):
    # Set the vectors (Python List) with these hours for the forcing data
    # Air pressure at the measurement height
    airpres = [0 for x in range(H)] #[Pa]
    # Air temperature at the measurement height
    airtemp = [0 for x in range(H)]#[K]
    # Downward longwave radiation at the upper boundary
    LWRadAtm = [0 for x in range(H)] #[W m-2] 
    # Precipitation rate
    pptrate = [0 for x in range(H)] #[kg m-2 s-1]
    # Specific humifity at the measurement height
    spechum = [0 for x in range(H)] #[g g-1]
    # Downward shortwave radiation at the upper boundary
    SWRadAtm = [0 for x in range(H)] #[W m-2]
    # Observation time
    time = [0 for x in range(H)] #[days since 1979-01-01 00:00:00]
    #wind speed at the measurement height
    windspd = [0 for x in range(H)] #[m s-1]

    F = {'airpres':airpres, 'airtemp':airtemp, 'pptrate':pptrate, 'spechum':spechum, 'windspd':windspd}
    
    return F

In [248]:
def extractGrib(g, lon_idx, lat_idx, verbose=False):

    # Return value of "MASKED" if masked
    if np.ma.is_masked(g[11].values[lat_idx, lon_idx]):
        return {'airtemp':-9999, 'spechum':-9999, 'airpres':-9999, 'forcingUGRD':-9999, 
       'windspd':-9999, 'LWRadAtm':-9999, 'forcingCONVfrac':-9999, 
       'forcingCAPE':-9999,'forcingPEVAP':-9999, 'pptrate':-9999, 'SWRadAtm':-9999}
    
    # 1:11:11 TMP, 2-m above ground Temperature [K]
    airtemp = g[1].values[lat_idx, lon_idx] 
    # 2:51:51 SPFH, 2-m above ground Specific humidity [kg/kg]
    spechum = g[2].values[lat_idx, lon_idx] 
    # 3:1:1 PRES, Surface pressure [Pa]
    airpres = g[3].values[lat_idx, lon_idx] 
    # 4:33:33 UGRD, 10-m above ground Zonal wind speed [m/s]
    forcingUGRD = g[4].values[lat_idx, lon_idx]
    # 5:34:34 VGRD, 10-m above ground Meridonal wind speed [m/s]
    windspd = g[5].values[lat_idx, lon_idx] 
    # 6:205:205 DLWRF,  Longwave radiation flux downwards [W/m^2]
    LWRadAtm = g[6].values[lat_idx, lon_idx]
    # 7:153:153 CONVfrac, Frac of total precip convective
    forcingCONVfrac = g[7].values[lat_idx, lon_idx] 
    # 8:157:157 CAPE, 180-mb above ground Convective Available Potential Energy
    forcingCAPE = g[8].values[lat_idx, lon_idx] 
    
    # PEVAP, Potential evaporation hourly total   MAYBE: Adiabatic tendency of temperature?
    forcingPEVAP = g[9].values[lat_idx, lon_idx]
    
    # 10:61:61 APCP, Precipitation hourly total [kg/m^2/hr]
    pptrate = g[10].values[lat_idx, lon_idx] / 60 / 60
    
    # 11:204:204 DSWRF, Shortwave radiation flux downwards (surface) [W/m^2]
    SWRadAtm = g[11].values[lat_idx, lon_idx]
            
    G={'airtemp':airtemp, 'spechum':spechum, 'airpres':airpres, 'forcingUGRD':forcingUGRD, 
       'windspd':windspd, 'LWRadAtm':LWRadAtm, 'forcingCONVfrac':forcingCONVfrac, 
       'forcingCAPE':forcingCAPE,'forcingPEVAP':forcingPEVAP, 'pptrate':pptrate, 'SWRadAtm':SWRadAtm}
    
    if verbose:
        print("TMP, 2-m above ground Temperature [K]: {}".format(airtemp))
        print("SPFH, 2-m above ground Specific humidity [kg/kg]: {}".format(spechum))
        print("PRES, Surface pressure [Pa]: {}".format(airpres))
        print("UGRD, 10-m above ground Zonal wind speed [m/s]: {}".format(forcingUGRD))
        print("VGRD, 10-m above ground Meridonal wind speed [m/s]: {}".format(windspd))
        print("DLWRF,  Longwave radiation flux downwards (surface) [W/m^2]: {}".format(LWRadAtm))
        print("CONVfrac, Fraction of total precipitation that is convective: {}".format(forcingCONVfrac))
        print("CAPE, 180-mb above ground Convective Available Potential Energy: {}".format(forcingCAPE))
        print("PEVAP, Potential evaporation hourly total: {}".format(forcingPEVAP))
        print("APCP, Precipitation hourly total [kg/m^2/hr]: {}".format(pptrate))
        print("DSWRF, Shortwave radiation flux downwards (surface) [W/m^2]: {}".format(SWRadAtm))
        
    return G

In [169]:
count = 0
for x in range(464):
    for y in range(224):
        if waterMask[y,x]==1:
            count+=1
print(count)

75958


In [204]:
lats.reshape(224,464).shape

(224, 464)

In [203]:
lons.reshape(224,464)

array([[-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063],
       [-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063],
       [-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063],
       ...,
       [-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063],
       [-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063],
       [-124.938, -124.813, -124.688, ...,  -67.313,  -67.188,  -67.063]])

In [250]:
extractGrib(gbf, 400, 200, verbose=True)

TMP, 2-m above ground Temperature [K]: 266.75
SPFH, 2-m above ground Specific humidity [kg/kg]: 0.0022709000000000006
PRES, Surface pressure [Pa]: 96939.23
UGRD, 10-m above ground Zonal wind speed [m/s]: -1.6400000000000001
VGRD, 10-m above ground Meridonal wind speed [m/s]: -2.24
DLWRF,  Longwave radiation flux downwards (surface) [W/m^2]: 279.53000000000003
CONVfrac, Fraction of total precipitation that is convective: 0.0
CAPE, 180-mb above ground Convective Available Potential Energy: 0.0
PEVAP, Potential evaporation hourly total: 0.0178
APCP, Precipitation hourly total [kg/m^2/hr]: 7.183333333333332e-05
DSWRF, Shortwave radiation flux downwards (surface) [W/m^2]: 0.0


{'airtemp': 266.75,
 'spechum': 0.0022709000000000006,
 'airpres': 96939.23,
 'forcingUGRD': -1.6400000000000001,
 'windspd': -2.24,
 'LWRadAtm': 279.53000000000003,
 'forcingCONVfrac': 0.0,
 'forcingCAPE': 0.0,
 'forcingPEVAP': 0.0178,
 'pptrate': 7.183333333333332e-05,
 'SWRadAtm': 0.0}

In [251]:
extractGrib(gbf, 0, 0, verbose=True)

{'airtemp': -9999,
 'spechum': -9999,
 'airpres': -9999,
 'forcingUGRD': -9999,
 'windspd': -9999,
 'LWRadAtm': -9999,
 'forcingCONVfrac': -9999,
 'forcingCAPE': -9999,
 'forcingPEVAP': -9999,
 'pptrate': -9999,
 'SWRadAtm': -9999}

In [236]:
gbf[9].values[100,100]

0.0021000000000000003

In [228]:
lon_idx

-99